In [ ]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')

In [ ]:
data = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
data.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
data.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
data

In [ ]:
lats = []
lons = []
for i in data['Postal Code']:
    lats.append(nomi.query_postal_code(i)['latitude'])
    lons.append(nomi.query_postal_code(i)['longitude'])
data['lat'] = lats
data['lon'] = lons
data['geometry'] = geopandas.points_from_xy(data['lat'], data['lon'])
data

In [ ]:
fig = go.Figure(data=go.Scattergeo(
    lat = data['lat'],
    lon = data['lon'],
    text = data['Member']
)
               )

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig.show()
#fig.write_html('membermap.html')

## Finding Scale distances on a 4x8 sheet of plywood

In [ ]:
'''
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs
'''
x_range = [data['lon'].max(), data['lon'].min()]
y_range = [data['lat'].max(), data['lat'].min()]
data['x'] = (data['lon'] - x_range[1]) / (x_range[0] - x_range[1])
data['y'] = (data['lat'] - y_range[1]) / (y_range[0] - y_range[1])
data['x inches'] = data['x'] * 46
data['y inches'] = data['y'] * 46
data

In [ ]:
import plotly.express as px
px.scatter(data, x='x inches', y='y inches', hover_name='Member', hover_data=['Address', 'City', 'Postal Code'], height=800, width=400).show()
fig.show()